In [1]:
%cd ~/LevSeq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from levseq.seqfit import process_plate_files, gen_seqfitvis

In [4]:
# process the seq and fit to merge the two files
processed_plate_df, seqfit_path = process_plate_files(product=["pdt"], input_csv="tests/seqfit/HMC0225_HMC0226.csv")

In [ ]:
gen_seqfitvis(seqfit_path=seqfit_path)